In [42]:
import yfinance as yf
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import os
import sqlite3
from sqlite3 import Error
from bs4 import BeautifulSoup
import requests
import datetime
from datetime import timedelta
from twython import Twython
import seaborn as sns
import matplotlib.pyplot as plt
from math import sqrt
from statistics import mean, stdev
import plotly.express as px
import webbrowser
webbrowser.register('chrome', None, webbrowser.BackgroundBrowser('C:\\Program Files (x86)\\Google\\Chrome\\Application\\chrome.exe'))

In [23]:
symbols = ['ACB', 'F', 'GE', 'DIS', 'AAL', 'GPRO', 'DAL', 'MSFT', 'CCL', 'AAPL', 'FIT', 'SNAP', 'PLUG', 'BAC', 'BA', 'NCLH', 'INO', 'UAL', 'UBER', 'CGC', 'TSLA', 'AMD', 'CRON', 'RCL', 'TWTR', 'GRPN', 'FB', 'SBUX', 'MRO', 'ZNGA', 'BABA', 'T', 'KO', 'APHA', 'USO', 'XOM', 'AMZN', 'MFA', 'JBLU', 'NIO', 'MRNA', 'LUV', 'GM', 'GILD', 'MGM', 'SAVE', 'NFLX', 'NRZ', 'SPCE', 'LK', 'VSLR', 'AMC', 'PENN', 'VOO', 'TLRY', 'HAL', 'NOK', 'NVDA', 'CPRX', 'LYFT', 'SQ', 'SPY', 'V', 'NKE', 'SIRI', 'UCO', 'WORK', 'CPE', 'BYND', 'KOS', 'ET', 'OXY', 'PFE', 'ZM', 'CRBP', 'SPHD', 'FCEL', 'VKTX', 'JPM', 'NTDOY', 'NYMT', 'BP', 'ATVI', 'CSCO', 'WFC', 'WMT', 'GOOGL', 'INTC', 'GLUU', 'AUY', 'VTI', 'ERI', 'TXMD', 'SNE', 'PTON', 'ROKU', 'JNJ', 'IVR', 'MU']

In [27]:
"""Look at week long spans"""
start_date = datetime.date(2020, 4, 1)
end_date =  datetime.date(2020, 7, 16)
df = pd.DataFrame()
for i in symbols:
    tckr = yf.Ticker(i)
    results = tckr.history(start=start_date, end = end_date)
    results.drop(['Dividends', 'Stock Splits'], axis=1, inplace=True)
    results['Symbol'] =[i]* int(results.shape[0])
    results.reset_index(inplace = True)
    df = df.append(results, ignore_index=True)
    print(i, 'dataframe added to base.')
print('Complete')

ACB dataframe added to base.
F dataframe added to base.
GE dataframe added to base.
DIS dataframe added to base.
AAL dataframe added to base.
GPRO dataframe added to base.
DAL dataframe added to base.
MSFT dataframe added to base.
CCL dataframe added to base.
AAPL dataframe added to base.
FIT dataframe added to base.
SNAP dataframe added to base.
PLUG dataframe added to base.
BAC dataframe added to base.
BA dataframe added to base.
NCLH dataframe added to base.
INO dataframe added to base.
UAL dataframe added to base.
UBER dataframe added to base.
CGC dataframe added to base.
TSLA dataframe added to base.
AMD dataframe added to base.
CRON dataframe added to base.
RCL dataframe added to base.
TWTR dataframe added to base.
GRPN dataframe added to base.
FB dataframe added to base.
SBUX dataframe added to base.
MRO dataframe added to base.
ZNGA dataframe added to base.
BABA dataframe added to base.
T dataframe added to base.
KO dataframe added to base.
APHA dataframe added to base.
USO dat

In [30]:
df.drop('Date_2', axis = 1,inplace=True)
df.head()


,Date,Open,High,Low,Close,Volume,Symbol
0,2020-04-01,10.50,10.90,9.36,9.71,2614700,ACB
1,2020-04-02,9.96,11.02,9.61,10.00,2770100,ACB
2,2020-04-03,10.40,10.44,9.48,9.72,2549100,ACB
3,2020-04-06,10.14,10.37,9.36,9.60,4244600,ACB
4,2020-04-07,10.08,10.21,9.53,9.60,2423000,ACB


In [34]:
def daily_mean(x):
    values = [x['Open'], x['High'], x['Low'], x['Close']]
    return mean(values)
def daily_stddev(x):
    values = [x['Open'], x['High'], x['Low'], x['Close']]
    return stdev(values)
def daily_pct_change(x):
    return (x['Close']-x['Open'])/x['Open']
df['Mean'] = df.apply(lambda x: daily_mean(x), axis=1)
df['Std_Dev'] = df.apply(lambda x: daily_stddev(x), axis=1)
df['Pct_Change'] = df.apply(lambda x: daily_pct_change(x), axis=1)

In [37]:
df.head()

,Date,Open,High,Low,Close,Volume,Symbol,Mean,Std_Dev,Pct_Change
0,2020-04-01,10.50,10.90,9.36,9.71,2614700,ACB,10.1175,0.706747,-0.075238
1,2020-04-02,9.96,11.02,9.61,10.00,2770100,ACB,10.1475,0.607474,0.004016
2,2020-04-03,10.40,10.44,9.48,9.72,2549100,ACB,10.0100,0.483735,-0.065385
3,2020-04-06,10.14,10.37,9.36,9.60,4244600,ACB,9.8675,0.467574,-0.053254
4,2020-04-07,10.08,10.21,9.53,9.60,2423000,ACB,9.8550,0.340245,-0.047619


In [44]:
temp_x = df[df['Symbol']=='INTC']['Pct_Change']
temp_y = df[df['Symbol']=='NVDA']['Pct_Change']
fig = px.scatter(x = temp_x, y = temp_y)
fig.show(renderer='chrome')

In [242]:
help(yf)

Help on package yfinance:

NAME
    yfinance

DESCRIPTION
    # -*- coding: utf-8 -*-
    #
    # Yahoo! Finance market data downloader (+fix for Pandas Datareader)
    # https://github.com/ranaroussi/yfinance
    #
    # Copyright 2017-2019 Ran Aroussi
    #
    # Licensed under the Apache License, Version 2.0 (the "License");
    # you may not use this file except in compliance with the License.
    # You may obtain a copy of the License at
    #
    #     http://www.apache.org/licenses/LICENSE-2.0
    #
    # Unless required by applicable law or agreed to in writing, software
    # distributed under the License is distributed on an "AS IS" BASIS,
    # WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    # See the License for the specific language governing permissions and
    # limitations under the License.
    #

PACKAGE CONTENTS
    base
    multi
    shared
    ticker
    tickers
    utils

CLASSES
    builtins.object
        yfinance.tickers.Tickers
   

In [16]:
tckr = yf.Ticker('NFLX')
results = tckr.history(start = datetime.date(2020, 5, 23), end = datetime.date(2020, 6, 24))
results.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-05-26,427.77,428.50,413.14,414.77,7881100,0,0
2020-05-27,410.38,420.02,397.86,419.89,10446300,0,0
2020-05-28,417.24,422.37,411.50,413.44,5655100,0,0
2020-05-29,417.46,420.30,411.85,419.73,5270500,0,0
2020-06-01,418.83,426.72,415.98,425.92,3743700,0,0


In [26]:
print(timedelta(days=2).total_seconds())

172800.0


In [42]:
#Goal1: using opening value as mean, what is the variance throughout a single day.
def mean(_list):
    length = 1 if len(_list) == 0 else len(_list)
    return sum(_list)/length
def time_variance(symbol, window):
    start_date = datetime.date(2019, 5, 30)
    stop_date = datetime.date(2020, 5, 30)
    total_diff  = (stop_date-start_date)
    total_diff = total_diff.total_seconds()
    sum_errors = []
    while start_date != stop_date:
        tckr = yf.Ticker(symbol)
        window_start = start_date
        window_end = start_date+window
        window_prices = []
        skip = False
        while window_start != window_end:
            if len(tckr.history(start=window_start-timedelta(days=1), end=window_start)['Open']) ==1:
                open_price =float(tckr.history(start=window_start-timedelta(days=1), end=window_start)['Open'])
                close_price=float(tckr.history(start=window_start-timedelta(days=1), end=window_start)['Close'])
                window_prices.append(open_price)
                window_prices.append(close_price)
            window_start += timedelta(days=1)
        my_mean = mean(window_prices)
        #print(window_prices)
        _error = []
        for i in window_prices:
            _error.append((i-my_mean)**2)
        #print(_error)
        sum_errors.append(sqrt(mean(_error)))
        start_date += timedelta(days=1)
        time_remaining = stop_date - start_date
        time_remaining = time_remaining.total_seconds()
        percent_complete = ((total_diff-time_remaining)/total_diff)*100
        print(f'{percent_complete} percent complete', end='\r', flush = True)
    return mean(sum_errors)
print(time_variance('NFLX', timedelta(days=7)))


69.39890710382514 percent complete- NFLX: No data found for this date range, symbol may be delisted
- NFLX: No data found for this date range, symbol may be delisted
69.67213114754098 percent complete- NFLX: No data found for this date range, symbol may be delisted
- NFLX: No data found for this date range, symbol may be delisted
69.94535519125684 percent complete- NFLX: No data found for this date range, symbol may be delisted
- NFLX: No data found for this date range, symbol may be delisted
70.21857923497268 percent complete- NFLX: No data found for this date range, symbol may be delisted
- NFLX: No data found for this date range, symbol may be delisted
70.49180327868852 percent complete- NFLX: No data found for this date range, symbol may be delisted
- NFLX: No data found for this date range, symbol may be delisted
- NFLX: No data found for this date range, symbol may be delisted
70.76502732240438 percent complete- NFLX: No data found for this date range, symbol may be delisted
- N

In [29]:
database = pd.DataFrame()
lead_time = 1 #1 day lead time of info to not 
#columns DATE SYMBOL YESTERDAY_SENTIMENT(from twitter) CLOSE PRICE

[]


In [49]:
#Twitter API
#credentials
consumer_key = ''
consumer_secret = ''
python_tweets = Twython(consumer_key, consumer_secret)
query = {'q':'SYMBOL',
         'result_type':'popular',
         'count': 100,
         'lang': 'en',}

Date
2019-05-14    9.6747
Name: Open, dtype: float64

In [ ]:
def process_tweets(tweets):
    pass

In [ ]:
#article headlines


In [43]:
_date = []
_symbol = []
#_words = []
_open_price = []
_close_price =[]
start_date = datetime.date(2019, 6, 1)
end_date =  datetime.date(2020, 6, 1)
increment = timedelta(days=1)
loading_total = end_date-start_date
loading_total = loading_total.total_seconds() 
while start_date != end_date:
    for symbol in symbols:
        tckr = yf.Ticker(symbol)
        if start_date.weekday()<5: 
            _date.append(start_date)
            _symbol.append(symbol)
            _close = tckr.history(start=start_date, end=start_date+increment)['Close']
            _open = tckr.history(start=start_date, end=start_date+increment)['Open']
            _open_price.append(_open)
            _close_price.append(_close)
    loading_elapsed=end_date-start_date
    loading_elapsed =loading_total - loading_elapsed.total_seconds()
    percent_done = 100*loading_elapsed/loading_total
    print('{0:.2f} percent complete'.format(percent_done), end='\r', flush=True)
    #for each symbol get todays close price
    #Get twitter stuff

    start_date= start_date + increment

database['DATE'] = _date
database['SYMBOL'] = _symbol
#database['WORDS'] = _words
database['OPEN_PRICE'] = _open_price
database['CLOSE_PRICE'] = _close_price

rcent complete- BRK.B: No data found, symbol may be delisted
- BRK.B: No data found, symbol may be delisted
- PTON: Data doesn't exist for startDate = 1564635600, endDate = 1564722000
- PTON: Data doesn't exist for startDate = 1564635600, endDate = 1564722000
16.67 percent complete- BRK.B: No data found, symbol may be delisted
- BRK.B: No data found, symbol may be delisted
- PTON: Data doesn't exist for startDate = 1564722000, endDate = 1564808400
- PTON: Data doesn't exist for startDate = 1564722000, endDate = 1564808400
16.94 percent complete17.21 percent complete17.49 percent complete- BRK.B: No data found, symbol may be delisted
- BRK.B: No data found, symbol may be delisted
- PTON: Data doesn't exist for startDate = 1564981200, endDate = 1565067600
- PTON: Data doesn't exist for startDate = 1564981200, endDate = 1565067600
17.76 percent complete- BRK.B: No data found, symbol may be delisted
- BRK.B: No data found, symbol may be delisted
- PTON: Data doesn't exist for startDate = 1

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
database.head()

In [ ]:
database.to_csv('C:\Users\StewaJeb\AppData\Local\Programs\Python\Python37\my_code\Tensorflow_Sandbox\practice.csv')

In [19]:
#Create new SQLite Database
def create_connection(db_filepath):
    conn = None
    try:
        conn = sqlite3.connect(db_filepath)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()
create_connection(r'C:\Users\StewaJeb\AppData\Local\Programs\Python\Python37\my_code\Tensorflow_Sandbox\stp_db.db')

In [21]:
print([3]*10)

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
